# W_GL_BALANCE_FS ETL Process
### Oracle EBS GL_BALANCES to EDW W_GL_BALANCE_FS (Fact Staging)
**ODI Package:** SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT  
**LKM:** BIAPPS SQL to Oracle (Multi Transport)  
**IKM:** BIAPPS Oracle Control Append

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Variable LAST_EXTRACT_DATE - Get last extraction date
CREATE OR REPLACE TEMP VIEW VAR_LAST_EXTRACT_DATE AS
SELECT
  COALESCE(
    date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'),
    '1970-01-01 00:00:00'
  ) AS LAST_EXTRACT_DATE
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 3: Drop and Create Work Table C$_2623812_2_0 (as temp view)
DROP VIEW IF EXISTS C$_2623812_2_0;

CREATE OR REPLACE TEMP VIEW C$_2623812_2_0 AS
SELECT
  CAST(SQ_GL_BALANCES.CODE_COMBINATION_ID AS STRING) AS GL_ACCOUNT_ID,
  SQ_GL_BALANCES.END_DATE AS BALANCE_DATE,
  SQ_GL_BALANCES.BALANCE_TYPE_INDICATOR AS DB_CR_IND,
  -- Balance calculations with debit/credit logic
  CASE
    WHEN SQ_GL_BALANCES.BALANCE_TYPE_INDICATOR = 'CREDIT'
    THEN -(SQ_GL_BALANCES.BEGIN_BALANCE_CR + SQ_GL_BALANCES.PERIOD_NET_CR)
    WHEN SQ_GL_BALANCES.BALANCE_TYPE_INDICATOR = 'DEBIT'
    THEN (SQ_GL_BALANCES.BEGIN_BALANCE_DR + SQ_GL_BALANCES.PERIOD_NET_DR)
  END AS BALANCE_ACCT_AMT,
  CASE
    WHEN SQ_GL_BALANCES.BALANCE_TYPE_INDICATOR = 'CREDIT'
    THEN -SQ_GL_BALANCES.PERIOD_NET_CR
    WHEN SQ_GL_BALANCES.BALANCE_TYPE_INDICATOR = 'DEBIT'
    THEN SQ_GL_BALANCES.PERIOD_NET_DR
  END AS ACTIVITY_ACCT_AMT,
  SQ_GL_BALANCES.CURRENCY_CODE AS ACCT_CURR_CODE,
  SQ_GL_BALANCES.LEDGER_CURRENCY_CODE AS LOC_CURR_CODE,
  SQ_GL_BALANCES.ADJUSTMENT_PERIOD_FLAG AS ADJUSTMENT_FLAG,
  SQ_GL_BALANCES.ACTUAL_FLAG AS BALANCE_TYPE_FLAG,
  SQ_GL_BALANCES.TRANSLATED_FLAG,
  SQ_GL_BALANCES.LAST_UPDATE_DATE_BAL AS CHANGED_ON_DT,
  SQ_GL_BALANCES.LAST_UPDATE_DATE_LED AS AUX1_CHANGED_ON_DT,
  SQ_GL_BALANCES.LAST_UPDATE_DATE_PERIODS AS AUX2_CHANGED_ON_DT,
  CAST(SQ_GL_BALANCES.LEDGER_ID AS STRING) AS LEDGER_ID,
  SQ_GL_BALANCES.PERIOD_SET_NAME || '~' || SQ_GL_BALANCES.PERIOD_NAME AS PERIOD_ID,
  SQ_GL_BALANCES.INTEGRATION_ID
FROM (
  SELECT
    GL_BALANCES.LEDGER_ID,
    GL_BALANCES.CODE_COMBINATION_ID,
    GL_BALANCES.CURRENCY_CODE,
    GL_BALANCES.PERIOD_NAME,
    GL_BALANCES.ACTUAL_FLAG,
    COALESCE(GL_BALANCES.TRANSLATED_FLAG, 'N') AS TRANSLATED_FLAG,
    GL_BALANCES.PERIOD_NET_DR,
    GL_BALANCES.PERIOD_NET_CR,
    GL_BALANCES.BEGIN_BALANCE_DR,
    GL_BALANCES.BEGIN_BALANCE_CR,
    GL_BALANCES.LAST_UPDATE_DATE AS LAST_UPDATE_DATE_BAL,
    GL_PERIODS.PERIOD_SET_NAME,
    GL_PERIODS.END_DATE,
    GL_PERIODS.ADJUSTMENT_PERIOD_FLAG,
    GL_PERIODS.LAST_UPDATE_DATE AS LAST_UPDATE_DATE_PERIODS,
    GL_LEDGERS.CURRENCY_CODE AS LEDGER_CURRENCY_CODE,
    GL_LEDGERS.LAST_UPDATE_DATE AS LAST_UPDATE_DATE_LED,
    BT.BALANCE_TYPE_INDICATOR,
    -- Integration ID pattern for Fact
    concat('GL~', CAST(GL_BALANCES.LEDGER_ID AS STRING), '~',
           CAST(GL_BALANCES.CODE_COMBINATION_ID AS STRING), '~',
           GL_BALANCES.PERIOD_NAME, '~', BT.BALANCE_TYPE_INDICATOR, '~',
           GL_BALANCES.ACTUAL_FLAG, '~', COALESCE(GL_BALANCES.TRANSLATED_FLAG, 'N')) AS INTEGRATION_ID
  FROM workspace.oracle_ebs.GL_BALANCES GL_BALANCES
  INNER JOIN workspace.oracle_ebs.GL_PERIODS GL_PERIODS
    ON GL_BALANCES.PERIOD_NAME = GL_PERIODS.PERIOD_NAME
    AND GL_BALANCES.PERIOD_TYPE = GL_PERIODS.PERIOD_TYPE
  INNER JOIN workspace.oracle_ebs.GL_LEDGERS GL_LEDGERS
    ON GL_BALANCES.LEDGER_ID = GL_LEDGERS.LEDGER_ID
    AND GL_PERIODS.PERIOD_SET_NAME = GL_LEDGERS.PERIOD_SET_NAME
  CROSS JOIN (
    SELECT 'CREDIT' AS BALANCE_TYPE_INDICATOR
    UNION ALL
    SELECT 'DEBIT' AS BALANCE_TYPE_INDICATOR
  ) BT
  WHERE GL_BALANCES.ACTUAL_FLAG IN ('A', 'E')
    AND GL_BALANCES.TEMPLATE_ID IS NULL
    AND (GL_BALANCES.LAST_UPDATE_DATE > to_timestamp((SELECT LAST_EXTRACT_DATE FROM VAR_LAST_EXTRACT_DATE), 'yyyy-MM-dd HH:mm:ss')
         OR GL_LEDGERS.LAST_UPDATE_DATE > to_timestamp((SELECT LAST_EXTRACT_DATE FROM VAR_LAST_EXTRACT_DATE), 'yyyy-MM-dd HH:mm:ss')
         OR GL_PERIODS.LAST_UPDATE_DATE > to_timestamp((SELECT LAST_EXTRACT_DATE FROM VAR_LAST_EXTRACT_DATE), 'yyyy-MM-dd HH:mm:ss'))
) SQ_GL_BALANCES

In [ ]:
%sql
-- Step 4: Insert new rows into W_GL_BALANCE_FS from work table
INSERT INTO workspace.oracle_edw.W_GL_BALANCE_FS
(
  GL_ACCOUNT_ID,
  BALANCE_DATE,
  DB_CR_IND,
  BALANCE_ACCT_AMT,
  ACTIVITY_ACCT_AMT,
  ACCT_CURR_CODE,
  LOC_CURR_CODE,
  ADJUSTMENT_FLAG,
  BALANCE_TYPE_FLAG,
  TRANSLATED_FLAG,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  LEDGER_ID,
  PERIOD_ID,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM
)
SELECT
  GL_ACCOUNT_ID,
  BALANCE_DATE,
  DB_CR_IND,
  BALANCE_ACCT_AMT,
  ACTIVITY_ACCT_AMT,
  ACCT_CURR_CODE,
  LOC_CURR_CODE,
  ADJUSTMENT_FLAG,
  BALANCE_TYPE_FLAG,
  TRANSLATED_FLAG,
  CAST(CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  CAST(AUX1_CHANGED_ON_DT AS TIMESTAMP) AS AUX1_CHANGED_ON_DT,
  CAST(AUX2_CHANGED_ON_DT AS TIMESTAMP) AS AUX2_CHANGED_ON_DT,
  LEDGER_ID,
  PERIOD_ID,
  380 AS DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  '1' AS TENANT_ID,
  '0' AS X_CUSTOM
FROM C$_2623812_2_0

In [ ]:
%sql
-- Step 5: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT' AS PACKAGE_NAME,
    'W_GL_BALANCE_FS' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2623812 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 6: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2623812 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 7: Drop work table (temp view)
DROP VIEW IF EXISTS C$_2623812_2_0;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL;
DROP VIEW IF EXISTS VAR_LAST_EXTRACT_DATE